# Document Search with LangChain

This example shows how to use the Python [LangChain](https://python.langchain.com/docs/get_started/introduction) library to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using the text stored in a collection of local PDF documents.

### <u>Requirements</u>
1. As you will accessing the LLMs and embedding models through Vector AI Engineering's Kaleidoscope Service (Vector Inference + Autoscaling), you will need to request a KScope API Key:

   Run the following command (replace ```<user_id>``` and ```<password>```) from **within the cluster** to obtain the API Key. The ```access_token``` in the output is your KScope API Key.
  ```bash
  curl -X POST -d "grant_type=password" -d "username=<user_id>" -d "password=<password>" https://kscope.vectorinstitute.ai/token
  ```
2. After obtaining the `.env` configurations, make sure to create the ```.kscope.env``` file in your home directory (```/h/<user_id>```) and set the following env variables:
- For local models through Kaleidoscope (KScope):
    ```bash
    export OPENAI_BASE_URL="https://kscope.vectorinstitute.ai/v1"
    export OPENAI_API_KEY=<kscope_api_key>
    ```
- For OpenAI models:
   ```bash
   export OPENAI_BASE_URL="https://api.openai.com/v1"
   export OPENAI_API_KEY=<openai_api_key>
   ```
3. (Optional) Upload some pdf files into the `source_documents` subfolder under this notebook. We have already provided some sample pdfs, but feel free to replace these with your own.

## Set up the RAG workflow environment

#### Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import requests
import sys

from pathlib import Path

from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

#### Load config files

In [10]:
# Add root folder of the rag_bootcamp repo to PYTHONPATH
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.insert(0, str(parent_dir))

from utils.load_secrets import load_env_file
load_env_file()

In [11]:
GENERATOR_BASE_URL = os.environ.get("OPENAI_BASE_URL")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

#### Set up some helper functions

In [12]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

#### Make sure other necessary items are in place

In [13]:
# Look for the source_documents folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source_documents"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

#### Choose LLM and embedding model

In [14]:
GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
EMBEDDING_MODEL_NAME = "BAAI/bge-base-en-v1.5"

## Start with a basic generation request without RAG augmentation

Let's start by asking Llama-3.1 a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's world knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is domain-specific and it won't know the answer to. A good example would be an obscure detail buried deep within a company's annual report. For example:

*How many Vector scholarships in AI were awarded in 2022?*

In [15]:
query = "How many Vector scholarships in AI were awarded in 2022?"

## Now send the query to the open source model using KScope

In [16]:
llm = ChatOpenAI(
    model=GENERATOR_MODEL_NAME,
    temperature=0,
    max_tokens=None,
    base_url=GENERATOR_BASE_URL,
    api_key=OPENAI_API_KEY
)
message = [
    ("human", query),
]
try:
    result = llm.invoke(message)
    print(f"Result: \n\n{result.content}")
except Exception as err:
    if "Error code: 503" in err.message:
        print(f"The model {GENERATOR_MODEL_NAME} is not ready yet.")
    else:
        raise

Result: 

I don't have access to real-time data or specific information about the number of Vector scholarships in AI awarded in 2022. For the most accurate and up-to-date information, I recommend checking directly with Vector Institute or their official website. They would have the most current details on their scholarship programs and awards.


Without additional information, Llama-3.1 is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source_documents` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from `source_documents`

Start by reading in all the PDF files from `source_documents`, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [116]:

from langchain.document_loaders import TextLoader

In [18]:
ls /projects/RAG2/scotia-2/Datasets-Scotia-2/IBIS

'11114CA Wheat Farming in Canada Industry Report.pdf'
'11115CA Corn Farming in Canada Industry Report.pdf'
'33639CA Auto Parts Manufacturing in Canada Industry Report.pdf'
'44111CA New Car Dealers in Canada Industry Report.pdf'
'48412CA Long-Distance Freight Trucking in Canada Industry Report.pdf'
'48422CA Local Specialized Freight Trucking in Canada Industry Report.pdf'
'48423CA Long-Distance Specialized Freight Trucking in Canada Industry Report.pdf'


In [90]:
model_kwargs = {'device': 'cuda', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

print(f"Setting up the embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name=   EMBEDDING_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

Setting up the embeddings model...


In [152]:
%%time
# Load the IBIS pdfs
#directory_path = "./source_documents"
directory_path = "/projects/RAG2/scotia-2/Datasets-Scotia-2/IBIS"
loader = PyPDFDirectoryLoader(directory_path)
docs = loader.load()
print(f"Number of source documents: {len(docs)}")

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=32)
chunks = text_splitter.split_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

Number of source documents: 280
Number of text chunks: 654
CPU times: user 9.77 s, sys: 17.3 ms, total: 9.78 s
Wall time: 9.77 s


In [153]:
#chunks2 = text_splitter.split_documents(document)

In [154]:
#len (chunks2)

In [155]:
# Adding Reuters data
file_paths = ['/projects/RAG2/scotia-2/Datasets-Scotia-2/Agriculture_txt/agri_ca_co.csv', 
              '/projects/RAG2/scotia-2/Datasets-Scotia-2/Transport_txt/transport_CA.csv', 
              '/projects/RAG2/scotia-2/Datasets-Scotia-2/Auto_txt/auto_ca.csv', 
             ]
def load_txt_file(file_path):
    # Create a TextLoader instance

    loader = TextLoader(file_path)

    # Load the document

    document = loader.load()
    chunks2 =text_splitter.split_documents(document)
    print(f"Number of text chunks: {len(chunks2)}")
    return chunks2

In [156]:
for file_path in file_paths:
    chunks2 = load_txt_file(file_path)
    chunks= chunks +chunks2

Number of text chunks: 1212
Number of text chunks: 927
Number of text chunks: 2065


#### Define the embeddings model

## Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query. (This takes about 1-2 minutes)

In [157]:
%%time
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Retrieve the most relevant context from the vector store based on the query


CPU times: user 1min 52s, sys: 485 ms, total: 1min 52s
Wall time: 1min 47s


In [174]:
industries  = ["auto dealer", "wheat farming", 'trucking']
industry  = "wheat farming"
industry ="auto dealer"
query = f"Where are most {industry} companies are located in Canada?"
query1  = f' Which countries are competitors for canadian {industry} companies internationally?' # bad

query2 = f'What is the profit margin of Canadian {industry}'

#query = f'Who are the main palyers in Canadian  {industry}?' # bad
query3  = ' Which countries are competitors for canadian industry internationally?' # bad

In [175]:
%%time
retrieved_docs = retriever.invoke(query1)

CPU times: user 27.2 ms, sys: 11.8 ms, total: 39 ms
Wall time: 35.8 ms


Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [170]:
#pretty_print_docs(retrieved_docs)

## Now send the query to the RAG pipeline

In [177]:
%%time
rag_pipeline = RetrievalQA.from_llm(llm=llm, retriever=retriever)
result = rag_pipeline.invoke(input=query)
print(f"Result: \n\n{result['result'].replace ('According to the provided context, ', '')}")

Result: 

most new car dealers in Canada are located in Ontario. This is due to several factors, including:

1. Population: Ontario contains the majority of Canada's population, contributing to above-average demand for new vehicles.
2. Major cities: Cities like Toronto and Ottawa contribute to demand, and the region also houses the majority of automobile and SUV manufacturers, allowing dealers to leverage proximity to key suppliers to reduce transportation costs and better allocate inventories to consumer demand.

Additionally, Quebec and British Columbia are also popular destinations for new car dealers in Canada, due to their relatively high population density and affluent consumers, as well as strong retail and tourism activity.
CPU times: user 54.4 ms, sys: 7.58 ms, total: 62 ms
Wall time: 4.26 s
